In [ ]:
# INstalação dos pacotes
!pip install pysal
!pip install contextily
!pip install xlsxwriter


# Importação de pacotes e diretórios

In [ ]:
import pandas as pd
import numpy as np
import pysal as ps
import geopandas as gpd
import matplotlib.pyplot as plt
from splot.mapping import vba_choropleth as maps
import matplotlib.pyplot as plt
from matplotlib import colors
import contextily
import spreg
import xlsxwriter
#from .sqlite import head_to_sql, start_sql
import os
import libpysal
from mlxtend.preprocessing import standardize
os.chdir('/content/drive/MyDrive/Dados_Cod')


In [ ]:
# Importando a base de dados (dentr do shp)
db = libpysal.io.open('mapa_regressao_vf.dbf','r')
shp = gpd.read_file('mapa_regressao_vf.shp')

In [ ]:
db.header

['ln_farming',
 'ln_area',
 'ln_corre',
 'dv_desp_ar',
 'ln_despes',
 'ln_VBP',
 'CD_GEOC',
 'Valor',
 'VBP17',
 'VBP06',
 'AGROTOX',
 'IRRIGAC',
 'AREA_AG',
 'ASSIST_',
 'BENS_EQ',
 'CORRECA',
 'CREDITO',
 'DESPESA',
 'TRABALH',
 'UF',
 'Nome_Mn',
 'ano',
 'Forest',
 'Farming',
 'Outros',
 'Percent',
 'tx_desf',
 'tx_Frst',
 'CD_MUNI',
 'RE',
 'Sl_Sdst',
 'Nrt_Nrd',
 'Nordest',
 'CAT_desf',
 'CAT_vbp',
 'LISA_CL',
 'LISA_biv']

In [ ]:
type(shp)
# Criando as variáveis
vr_fl = db.by_col("tx_desf")
y = np.array(vr_fl)
y.shape = (len(vr_fl),1)

#x_names = ['IRRIGAC', 'VBP17', #'AREA_AG',
#           'AGROTOX', 'ASSIST_', 'CREDITO', 'DESPESA', 'TRABALH', 'BENS_EQ']
x_names = ['VBP17', 'CORRECA',#'Farming',
           'DESPESA', 'TRABALH', 'Sl_Sdst','Nordest','CREDITO']


pondera = np.array(db.by_col("AREA_AG"))
pondera.shape = (len(pondera),1)



X = []
#X.append(db.by_col("IRRIGAC"))
X.append(np.log(db.by_col("VBP17")))
X.append(db.by_col("CORRECA"))
#X.append(np.log(np.array(db.by_col("Farming"))+1))
#X.append(db.by_col("AGROTOX"))
#X.append(db.by_col("ASSIST_"))
#X.append(db.by_col("CREDITO"))
X.append(db.by_col("DESPESA"))
X.append(db.by_col("TRABALH"))

X.append(db.by_col("Sl_Sdst"))
X.append(db.by_col("Nordest"))
X.append(np.log(np.array(db.by_col("CREDITO"))+1))

X = np.array(X).T

x = X/pondera

In [ ]:
# Criando a matriz de peso espacial
w = libpysal.weights.Queen.from_shapefile("mapa_regressao_vf.shp")
w.transform = 'r' # Padroniza na linha, de forma que a soma será 1

/usr/local/lib/python3.10/dist-packages/libpysal/cg/shapes.py:1492: FutureWarning: Objects based on the `Geometry` class will deprecated and removed in a future version of libpysal.
  warnings.warn(dep_msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/libpysal/cg/shapes.py:1208: FutureWarning: Objects based on the `Geometry` class will deprecated and removed in a future version of libpysal.
  warnings.warn(dep_msg, FutureWarning)


('WARNING: ', 465, ' is an island (no neighbors)')


/usr/local/lib/python3.10/dist-packages/libpysal/weights/weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
 There is 1 island with id: 465.
  warnings.warn(message)


In [ ]:
# Estima um OLS com os testes

# Normalizar a matriz
X0 = X#standardize(X)

ols = spreg.OLS(y,X0,w=w, name_x=x_names, name_y='des1785', name_w = 'Matriz Queen',name_ds='Dados_Desma',spat_diag=True)

print(ols.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            : Dados_Desma
Weights matrix      :Matriz Queen
Dependent Variable  :     des1785                Number of Observations:        1426
Mean dependent var  :      0.1693                Number of Variables   :           8
S.D. dependent var  :      0.1558                Degrees of Freedom    :        1418
R-squared           :      0.2269
Adjusted R-squared  :      0.2231
Sum squared residual:     26.7321                F-statistic           :     59.4700
Sigma-square        :       0.019                Prob(F-statistic)     :   5.526e-75
S.E. of regression  :       0.137                Log likelihood        :     812.027
Sigma-square ML     :       0.019                Akaike info criterion :   -1608.055
S.E of regression ML:      0.1369                Schwarz criterion     :   -1565.954

------------------------------------------------------------

## Teste de I-Moran

In [ ]:
m = spreg.diagnostics_sp.MoranRes(ols, w, z=True)
print("I-moran:",round(m.I,4),"   p-valor:",round(m.p_norm,4))

I-moran: 0.414    p-valor: 0.0


1

## Estimação do SAR

In [ ]:
# Estima um SAR
sar = spreg.ML_Lag(y,X0,w=w, name_x=x_names, name_y='des1785')
print(sar.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     des1785                Number of Observations:        1426
Mean dependent var  :      0.1693                Number of Variables   :           9
S.D. dependent var  :      0.1558                Degrees of Freedom    :        1417
Pseudo R-squared    :      0.4928
Spatial Pseudo R-squared:  0.2300
Log likelihood      :   1035.1637
Sigma-square ML     :      0.0125                Akaike info criterion :   -2052.327
S.E of regression   :      0.1118                Schwarz criterion     :   -2004.964

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
---------------------------------------------------------------

## Estimando o SAR GMM - COM Heterocedasticidade

In [ ]:

GM_SAR= spreg.GM_Lag(y,X0,  w=w,   w_lags=1,lag_q=True, robust='white',name_x=x_names,
                     name_y='des1785', name_w = 'Matriz Queen', name_ds='Desmatamento',
                              spat_diag=True,)
print(GM_SAR.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :Desmatamento
Weights matrix      :Matriz Queen
Dependent Variable  :     des1785                Number of Observations:        1426
Mean dependent var  :      0.1693                Number of Variables   :           9
S.D. dependent var  :      0.1558                Degrees of Freedom    :        1417
Pseudo R-squared    :      0.4732
Spatial Pseudo R-squared:  0.2315

White Standard Errors
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        -0.01488         0.03410        -0.43630         0.66262
               VBP17         0.01005         0.00449         2.23810         0.02521
             CORRECA

## Estima o SEM

In [ ]:
sem = spreg.ML_Error(y,X0,w=w, name_x=x_names, name_y='des1785', name_w = 'Matriz Queen',name_ds='Desmatamento')
print(sem.summary)

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:913: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ML SPATIAL ERROR (METHOD = full)
---------------------------------------------------
Data set            :Desmatamento
Weights matrix      :Matriz Queen
Dependent Variable  :     des1785                Number of Observations:        1426
Mean dependent var  :      0.1693                Number of Variables   :           8
S.D. dependent var  :      0.1558                Degrees of Freedom    :        1418
Pseudo R-squared    :      0.2260
Log likelihood      :   1029.6592
Sigma-square ML     :      0.0126                Akaike info criterion :   -2043.318
S.E of regression   :      0.1121                Schwarz criterion     :   -2001.217

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.05612    

## Estima o SEM GMM Considerando que os resíduos possuem heterocedasticidade

In [ ]:

GM_ERRO = spreg.GM_Error_Het(y,X0,w,name_y='des1785',name_x = x_names, name_w = 'Matriz Queen', name_ds='Desmatamento')

print(GM_ERRO.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: GM SPATIALLY WEIGHTED LEAST SQUARES (HET)
------------------------------------------------------------
Data set            :Desmatamento
Weights matrix      :Matriz Queen
Dependent Variable  :     des1785                Number of Observations:        1426
Mean dependent var  :      0.1693                Number of Variables   :           8
S.D. dependent var  :      0.1558                Degrees of Freedom    :        1418
Pseudo R-squared    :      0.2261
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.05310         0.04448         1.19371         0.23259
               VBP17         0.01555         0.00419    

In [ ]:
## Teste de Correlação espacial - I-Moran
m = spreg.diagnostics_sp.MoranRes(ols, w, z=True)
print("I-moran:",round(m.I,4),"   p-valor:",round(m.p_norm,4))

I-moran: 0.414    p-valor: 0.0


In [ ]:
spreg.diagnostics.vif(ols)

[None,
 (2.561727920263733, 0.3903615181338418),
 (2.0477814943338175, 0.4883333513692676),
 (1.9412422780955605, 0.5151340516759411),
 (2.5478591247297473, 0.39248637818861765),
 (1.3053946125561275, 0.7660518822288332),
 (2.080749040531534, 0.4805961605752065),
 (1.8357693715944199, 0.5447307355016335)]

In [ ]:
# VIF - Colinearidade = spreg.diagnostics.vif

# Normalidade de Jarque-Bera = spreg.diagnostics.jarque_bera

# Heterocedasticidade breusch-pagan = spreg.diagnostics.breusch_pagan

# LM teste spreg.diagnostics_sp.LMtests
LM_testes = spreg.diagnostics_sp.LMtests(ols,w)
# LM Erro (Estatística , p-valor)
print(round(LM_testes.lme[0],4), round(LM_testes.lme[1],4))
# LM Lag (Estatística , p-valor)
print(round(LM_testes.lml[0],4), round(LM_testes.lml[1],4))

553.2783 0.0
553.401 0.0


In [ ]:
# Construindo os impactos diretos e indiretos

# Convertendo para uma matrix de pesos
wf = w.full()[0]

N_total= GM_SAR.n
# Criando uma matriz Identidade
I = np.identity(N_total)

# Puxando o valor do rho
rho = GM_SAR.betas[8]


In [ ]:
print(GM_SAR.summary)
GM_SAR.betas[8]

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :Desmatamento
Weights matrix      :Matriz Queen
Dependent Variable  :     des1785                Number of Observations:        1426
Mean dependent var  :      0.1693                Number of Variables   :           9
S.D. dependent var  :      0.1558                Degrees of Freedom    :        1417
Pseudo R-squared    :      0.4732
Spatial Pseudo R-squared:  0.2315

White Standard Errors
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        -0.01488         0.03410        -0.43630         0.66262
               VBP17         0.01005         0.00449         2.23810         0.02521
             CORRECA

array([0.49958417])

In [ ]:
# Criando a primeira parte do cálculo do efeito marginal
#VErificar  cálculos em Lesege e peace
InvMatMFX = np.linalg.inv(np.subtract(I , rho*wf))
# Calculando para a variável "logp"
VBP17 =InvMatMFX*GM_SAR.betas[1]
CORRECA = InvMatMFX*GM_SAR.betas[2]
DESPESA = InvMatMFX*GM_SAR.betas[3]
TRABALH  = InvMatMFX*GM_SAR.betas[4]
CREDITO  = InvMatMFX*GM_SAR.betas[7]

In [ ]:
# Efeito direto
dir_VBP17 = np.diag(VBP17).mean()
dir_CORRECA = np.diag(CORRECA).mean()
dir_DESPESA = np.diag(DESPESA).mean()
dir_TRABA = np.diag(TRABALH).mean()
dir_CREDITO = np.diag(CREDITO).mean()

# Efeito total
tot_VBP17 = np.sum(VBP17)/N_total
tot_CORRECA = np.sum(CORRECA)/N_total
tot_DESPESA = np.sum(DESPESA)/N_total
tot_TRABA = np.sum(TRABALH)/N_total
tot_CREDITO = np.sum(CREDITO)/N_total

# Efeito indireto

ind_VBP17 = tot_VBP17 - dir_VBP17
ind_CORRECA= tot_CORRECA - dir_CORRECA
ind_DESPESA = tot_DESPESA - dir_DESPESA
ind_TRABA = tot_TRABA - dir_TRABA
ind_CREDITO = tot_CREDITO - dir_CREDITO

In [ ]:
pd.DataFrame(np.array([[dir_VBP17,  ind_VBP17,  tot_VBP17] ,
         [dir_CORRECA, ind_CORRECA, tot_CORRECA],
         [dir_DESPESA, ind_DESPESA, tot_DESPESA],
         [dir_TRABA, ind_TRABA, tot_TRABA]]
        ))

,0,1,2
0,1.070019e-02,9.370643e-03,2.007083e-02
1,-2.237499e-05,-1.959480e-05,-4.196978e-05
2,8.933356e-08,7.823347e-08,1.675670e-07
3,-4.971359e-06,-4.353645e-06,-9.325004e-06


In [ ]:

Vars = np.transpose(GM_SAR.name_z)
betas = np.transpose(GM_SAR.betas)
std = np.transpose(GM_SAR.std_err)


GM_SEM = GM_ERRO

resultados_GM_SAR = pd.concat([pd.DataFrame(betas,columns=Vars).T,
                               pd.DataFrame([GM_SAR.std_err],  columns=Vars).T], axis=1)


resultados_GM_SEM = pd.concat([pd.DataFrame(np.transpose(GM_SEM.betas),columns=Vars).T,
                               pd.DataFrame([GM_SEM.std_err],  columns=Vars).T], axis=1)

resultados_SAR = pd.concat([pd.DataFrame(np.transpose(sar.betas),columns=Vars).T,
                               pd.DataFrame([sar.std_err],  columns=Vars).T], axis=1)

resultados_SEM = pd.concat([pd.DataFrame(np.transpose(sem.betas),columns=Vars).T,
                               pd.DataFrame([sem.std_err],  columns=Vars).T], axis=1)

resultados_OLS = pd.concat([pd.DataFrame(np.transpose(ols.betas),columns=ols.name_x).T,
                               pd.DataFrame([ols.std_err],  columns=ols.name_x).T], axis=1)



final=[resultados_GM_SAR, resultados_GM_SEM, resultados_SAR, resultados_SEM,resultados_OLS]

nomes=['resultados_GM_SAR', 'resultados_GM_SEM', 'resultados_SAR', 'resultados_SEM','resultados_OLS']
writer = pd.ExcelWriter('/content/drive/MyDrive/Dados_Cod/Tabelas.xlsx', engine='xlsxwriter')
for i,df in enumerate(final, 0):
    df.to_excel(writer, sheet_name='%s' %nomes[i])

writer.save()


<ipython-input-40-fd55e6d2df6b>:33: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
